In [1]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings("ignore")

import re
from bs4 import BeautifulSoup
from tqdm import tqdm
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model, Sequential

Using TensorFlow backend.


In [2]:
def decontract(text):
    text = re.sub(r"won\'t", "will not", text)
    text = re.sub(r"can\'t", "can not", text)
    text = re.sub(r"n\'t", " not", text)
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"\'s", " is", text)
    text = re.sub(r"\'d", " would", text)
    text = re.sub(r"\'ll", " will", text)
    text = re.sub(r"\'t", " not", text)
    text = re.sub(r"\'ve", " have", text)
    text = re.sub(r"\'m", " am", text)
    return text

In [3]:
stop_words = set(stopwords.words('english'))
negative_stop_words = set(word for word in stop_words if "n't" in word or 'no' in word)
stop_words = stop_words - negative_stop_words

In [4]:
lemmatizer = WordNetLemmatizer()
def preprocess_text(review):
    review = re.sub(r"http\S+", "", review)             # removing website links
    review = BeautifulSoup(review, 'lxml').get_text()   # removing html tags
    review = decontract(review)                         # decontracting
    review = re.sub("\S*\d\S*", "", review).strip()     # removing the words with numeric digits
    review = re.sub('[^A-Za-z]+', ' ', review)          # removing non-word characters
    review = review.lower()                             # converting to lower case
    review = [word for word in review.split(" ") if not word in stop_words] # removing stop words
    review = [lemmatizer.lemmatize(token, "v") for token in review] #lemmatization
    review = " ".join(review)
    review.strip()
    return review

In [5]:
import keras
model = keras.models.load_model('/kaggle/input/model64/model.h5')

In [6]:
import pickle

with open('/kaggle/input/tokenizer/tokenizer.pkl', 'rb') as f:
    tokenizer = pickle.load(f)

In [7]:
max_review_length = 100
def predecir(text):
    data = {'Text': [text]}
    df = pd.DataFrame(data)
    df['Text'] = df['Text'].apply(lambda x: preprocess_text(x))
    x = tokenizer.texts_to_sequences(df['Text'].iloc[:1])
    xpredict = pad_sequences(x, maxlen=max_review_length)
    prediction = model.predict_classes(xpredict)
    return prediction[0] + 1

In [8]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 100, 100)          9784700   
_________________________________________________________________
bidirectional_1 (Bidirection (None, 200)               160800    
_________________________________________________________________
dense_1 (Dense)              (None, 5)                 1005      
Total params: 9,946,505
Trainable params: 9,946,505
Non-trainable params: 0
_________________________________________________________________


In [10]:
text = input()
predecir(text)

 I bought this to save money by cutting/slicing my own foods. I make my own bread and this enables me to have lovely thin sliced bread I buy my ham and cheese uncut/shaved/shredded and this enables me to slice/shave my own ham and cheese. Good money saver while not restricting myself on how I want my food prepared.


5